In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy.stats import linregress
import json
import pprint
import warnings
warnings.filterwarnings('ignore')

#use linregplot to easily generate a scatterplot with linear regression line and formula from a dataframe - linregplot(dataframe, column name for x, column name for y, x axis label, y axis label)
from scatterplot_generator import linregplot

# Import API key
from api_keys import api_key

In [ ]:
# read in csv
oscar_data= pd.read_csv("Resources/oscar_data.csv")
oscar_data.head()

In [ ]:
all_nominees = oscar_data[["Year", "Best Picture", "Nominee 1", "Nominee 2", "Nominee 3", "Nominee 4"]]

#Can we do all nominees in every category just for one year as well?

In [ ]:
base_url = "https://api.themoviedb.org/3/"

#Enter year

year = "2018"

startdate = f"{year}-01-01"
enddate = f"{year}-12-31"

all_movies_response = requests.get(f'{base_url}/discover/movie?include_adult=false&include_video=false&primary_release_date.gte={startdate}&primary_release_date.lte={enddate}&with_original_language=en&sort_by=vote_count.desc&api_key={api_key}').json()
total_pages = json.dumps(all_movies_response["total_pages"], indent=4, sort_keys=True)

In [ ]:
responseid = {'Title' : [],'ID' : [], 'Vote Average' : [], 'Release Date' : []}

#Request from API for each month
for i in range(1,int(total_pages)//10):

#EDIT DATES FOR DIFFERENT YEARS
    all_movies_response = requests.get(f'{base_url}/discover/movie?include_adult=false&include_video=false&primary_release_date.gte={startdate}&primary_release_date.lte={enddate}&with_original_language=en&sort_by=vote_count.desc&api_key={api_key}&page={i}').json()
    total_pages = json.dumps(all_movies_response["total_pages"], indent=4, sort_keys=True)

    #puts all results into a dictionary
    for j in range(len(all_movies_response["results"])):
        responseid['Title'].append(all_movies_response["results"][j]["title"])
        responseid['ID'].append(all_movies_response["results"][j]["id"])
        responseid['Vote Average'].append(all_movies_response["results"][j]["vote_average"])
        responseid['Release Date'].append(all_movies_response["results"][j]["release_date"])

#EDIT DATES FOR DIFFERENT YEARS

In [ ]:
#dictionary to dataframe, sorted and prepped for use

input_format = '%Y-%m-%d'
moviedata = pd.DataFrame(responseid)
# input string
moviedata['Release Date'] = pd.to_datetime(moviedata['Release Date'], utc=True)

moviedata['Release Date UNIX'] = moviedata['Release Date'].astype(np.int64)

moviedata = moviedata.sort_values("Release Date")
#moviedata['Release Date UNIX'] = (moviedata['Release Date'] - datetime(1970,1,1)).total_seconds()
moviedata['Day'] = moviedata['Release Date'].dt.day
moviedata['Month'] = moviedata['Release Date'].dt.month
moviedata['Year'] = moviedata['Release Date'].dt.year
moviedata = moviedata.reset_index(drop=True)

moviedata["Nominated"] = ""
moviedata

In [ ]:
titles = moviedata['Title'].to_list()
noms = []
print(year)


for movie in titles:
    mask = oscar_data[oscar_data.apply(lambda row: row.astype(str).str.contains(str(movie), case=False).any(), axis=1)]
    if mask.empty:
        noms.append("No")
        
    else:
        if mask['Year'].values[0] == year:
            noms.append("Yes")
        else:
            noms.append("No")

moviedata['Nominated'] = noms

In [ ]:
nominatedmovies = moviedata.loc[moviedata['Nominated'] == "Yes"]
nominatedmovies

In [ ]:
# how to get the winner/nominees for a given year. Could use this code or something like it later.

nominees = all_nominees.loc[all_nominees["Year"] == "2022",["Best Picture", "Nominee 1", "Nominee 2", "Nominee 3", "Nominee 4"]]


In [177]:
linregplot(moviedata, 'Release Date UNIX', 'Vote Average', 'Release Date', 'Average Review Score')


# scatter plot needs work.

NameError: name 'linregress' is not defined

In [ ]:
#requests.get(f'{base_url}/movie/1159307?&api_key={api_key}').json()
